# VacationPy
---



In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,st. john's,47.5649,-52.7093,16.66,96,100,8.23,CA,1718065324
1,1,timbuktu,16.7735,-3.0074,34.11,35,25,6.47,ML,1718065506
2,2,jamestown,42.0970,-79.2353,12.79,84,100,3.09,US,1718065251
3,3,college,64.8569,-147.8028,22.26,50,100,4.12,US,1718065506
4,4,maintirano,-18.0667,44.0167,20.97,83,13,2.09,MG,1718065507


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot =city_data_df.hvplot.points(
    "Lng",
    "Lat",
    c="Humidity", 
    s="Humidity", 
    geo=True, 
    xlabel='x', 
    ylabel='y', 
    tiles="OSM", color= "Humidity", colorbar=False, alpha=0.5,
    frame_width = 900, frame_height = 600,cmap="Viridis",
    hover_cols=['City', 'Humidity']
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df
ideal_weather = ideal_weather.loc[(ideal_weather["Max Temp"] <= 20) & (ideal_weather["Max Temp"] >= 15)]
ideal_weather = ideal_weather.loc[ideal_weather["Wind Speed"] <= 5]
ideal_weather = ideal_weather.loc[ideal_weather["Cloudiness"] == 0]



# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,94,east london,-33.0153,27.9116,15.96,49,0,3.92,ZA,1718065507
116,116,arraial do cabo,-22.9661,-42.0278,16.98,100,0,0.51,BR,1718065526
163,163,pisco,-13.7000,-76.2167,18.03,77,0,3.09,PE,1718065531
192,192,kapuskasing,49.4169,-82.4331,16.05,39,0,3.09,CA,1718065535
218,218,blackwater,-23.5833,148.8833,19.63,32,0,0.71,AU,1718065539
221,221,urangan,-25.3000,152.9000,19.00,68,0,4.92,AU,1718065539
239,239,petropavlovsk-kamchatsky,53.0452,158.6483,15.33,86,0,5.00,RU,1718065542
443,443,houlton,46.1262,-67.8403,16.23,80,0,1.54,US,1718065314
456,456,alpicat,41.6657,0.5556,18.31,81,0,0.00,ES,1718065570
466,466,port shepstone,-30.7414,30.4550,15.32,66,0,4.08,ZA,1718065571


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
94,east london,ZA,-33.0153,27.9116,49,
116,arraial do cabo,BR,-22.9661,-42.0278,100,
163,pisco,PE,-13.7000,-76.2167,77,
192,kapuskasing,CA,49.4169,-82.4331,39,
218,blackwater,AU,-23.5833,148.8833,32,
221,urangan,AU,-25.3000,152.9000,68,
239,petropavlovsk-kamchatsky,RU,53.0452,158.6483,86,
443,houlton,US,46.1262,-67.8403,80,
456,alpicat,ES,41.6657,0.5556,81,
466,port shepstone,ZA,-30.7414,30.4550,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
radius = 10000
params = {
    "radius": radius,
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
east london - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
kapuskasing - nearest hotel: Travelodge
blackwater - nearest hotel: No hotel found
urangan - nearest hotel: Ramada
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
houlton - nearest hotel: Shiretown Inn & Suites
alpicat - nearest hotel: Hotel Finca Prats Golf & SPA
port shepstone - nearest hotel: The Spot Backpackers'
iron river - nearest hotel: AmericInn
surgut - nearest hotel: Факел


,City,Country,Lat,Lng,Humidity,Hotel Name
94,east london,ZA,-33.0153,27.9116,49,No hotel found
116,arraial do cabo,BR,-22.9661,-42.0278,100,No hotel found
163,pisco,PE,-13.7000,-76.2167,77,La Portada
192,kapuskasing,CA,49.4169,-82.4331,39,Travelodge
218,blackwater,AU,-23.5833,148.8833,32,No hotel found
221,urangan,AU,-25.3000,152.9000,68,Ramada
239,petropavlovsk-kamchatsky,RU,53.0452,158.6483,86,ООО Постоялый двор
443,houlton,US,46.1262,-67.8403,80,Shiretown Inn & Suites
456,alpicat,ES,41.6657,0.5556,81,Hotel Finca Prats Golf & SPA
466,port shepstone,ZA,-30.7414,30.4550,66,The Spot Backpackers'


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", size="Humidity", c="Humidity",
                            geo=True, hover_cols=["City", "Humidity", "Hotel Name", "Country"],  
                            xlabel="x", ylabel="y", 
                            tiles="OSM", color = "Humidity", colorbar=False, alpha=0.8,
                            frame_width = 900, frame_height = 600, scale = 2, cmap="Viridis" ) 

# Display the map, 
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)